# Set up

In [ ]:
#Required Variables
ClearMapPath = 'path' #the location of the ClearMap2 folder
ProjectPath = 'path' # choose a parent folder to house your folder of images saved from this run of the program
folder ='FolderName'#name of folder for this experiment (set to None for automated folder named with today's date)
filename = 'filename.npy/tif' # set to your original npy or tif file
name = 'name' #appends the name to the end of all generated files


In [ ]:
#Optional Parameters

#Arterial Binarization Variables
minimum = 400 # minimum pixel value of vessels in your image - any pixel below this value will not be a vessel
maximum = 60000 # maximum pixel value of vessels in your image - any pixel above this value will be a vessel
equalize = False # set to True if you would like to turn equalize on
vesselize = False # set to True if you would like to turn vesselize on

#Vessel Filling Variables
filling_threshold = .5  #changes the probablity of a pixel being filled in (higher is more pixels)
# we recommend you set this to .5

#Graphing Variables


In [ ]:
# import required packages
import os
os.chdir(ClearMapPath)
from ClearMap.Environment import * 
%gui qt
from datetime import date
import shutil


In [ ]:
# create an output folder
today = date.today()
out = today.strftime("%m%d%y")
if folder is None:
    path = os.path.join(ProjectPath, out)
else:
    path = os.path.join(ProjectPath, folder)
os.makedirs(path)
print("The path to your images is", path)


In [1]:
# INITIALIZE WORKSPACE

directory = ProjectPath

expression_raw  = filename 

expression_arteries = filename 

expression_auto     = filename 

resources_directory = settings.resources_path

ws = wsp.Workspace('TubeMap', directory=directory);
ws.update(raw=expression_raw, arteries=expression_arteries, autofluorescence=expression_auto)
ws.info()

ws.filename('raw')

ws.file_list('raw')

s = ws.source('raw')


# converts tif to npy
if os.path.exists(os.path.join(ProjectPath, filename[-3] +'npy')):
    file.npy = filename[-3] +'npy'

else:
    
    if filename[-1] == 'y':
        file_npy = filename

    if filename[-1] == 'f':
        print("You inputed an tif file. Converting tif to npy.")
        io.convert_files(ws.file_list('raw', extension='tif'), extension='npy',
                     processes=12, verbose=True);
        filename = filename[:-3]
        file_npy = filename+'npy'
        print("Your file is now a npy named" + file_npy)

directory = ProjectPath

expression_raw  = file_npy 

expression_arteries = file_npy 

expression_auto = file_npy 

resources_directory = settings.resources_path
ws = wsp.Workspace('TubeMap', directory=directory);
ws.update(raw=expression_raw, arteries=expression_arteries, autofluorescence=expression_auto)
ws.info()

ws.filename('raw')

ws.file_list('raw')

s = ws.source('raw')

SyntaxError: invalid syntax (<ipython-input-1-7201c0ec8173>, line 3)

In [ ]:
# Use the below code if you would like to test the code on a smaller slice of your image
# DO NOT TEST ON LESS THAN 30 SLICES AT A TIME

# #%% Crop test data

# #select debug mode / test file name
# ws.debug = 'slice';

# #select sublice for testing the pipeline
# slicing = (slice(0,1080),slice(0,1280),slice(70,80));
# [ws.create_debug('raw', slicing=slicing)]


# # save slice image here as tif file 
# sink = ws.filename('raw')
# io.convert_files(sink, extension='tif', processes=12, verbose=True);


# Image Processing

In [ ]:
# ARTERIAL BINARIZATION

source = ws.filename('raw');
#threshold = 450 # 750 is recommended value for vasculature
sink   = ws.filename('binary', postfix='arteries' + name);
io.delete_file(sink)

binarization_parameter = vasc.default_binarization_parameter.copy();
binarization_parameter['clip']['clip_range'] = (minimum,maximum) #set based on min and max pixel values in the image (min, max)
# (400,60000) is recommended for vasculature

if not equalize:
    binarization_parameter['equalize'] = None;
if not vesselize:
    binarization_parameter['vesselize'] = None;

processing_parameter = vasc.default_binarization_processing_parameter.copy();
processing_parameter.update(processes = 20,
                            as_memory = True, verbose=True);

vasc.binarize(source, sink,
              binarization_parameter=binarization_parameter,
              processing_parameter=processing_parameter);


# save binary image here as tif file 
io.convert_files(sink, extension='tif', processes=12, verbose=True);


In [ ]:
#ARTERIAL BINARIZATION continued

source = ws.filename('binary', postfix='arteries' + name);
sink   = ws.filename('binary', postfix='arteries_postprocessed' + name);
sink_smooth = ws.filename('binary', postfix='arteries_smoothed' + name);

postprocessing_parameter = vasc.default_postprocessing_parameter.copy();
postprocessing_parameter['smooth'] = dict(iterations=6) #default is 6

postprocessing_processing_parameter = vasc.default_postprocessing_processing_parameter.copy();
postprocessing_processing_parameter.update(size_max = 50);

vasc.postprocess(source, sink, postprocessing_parameter=postprocessing_parameter,
                 processing_parameter=postprocessing_processing_parameter,
                 processes=None, verbose=True)

# save postprocessed binary image here
io.convert_files(sink, extension='tif', processes=None, verbose=True);


In [ ]:
# VESSEL FILLING

source = ws.filename('binary', postfix='arteries_postprocessed' + name);
sink   = ws.filename('binary', postfix='arteries_filled_threshold' + name);
io.delete_file(sink)

processing_parameter = vf.default_fill_vessels_processing_parameter.copy();
processing_parameter.update(size_max = 200,
                            size_min = 'fixed',
                            axes = all,
                            overlap = 50);

vf.fill_vessels(source, sink,
                resample=1, threshold= filling_threshold, cuda=None, 
                #default threshold is 0.5 - set above in Vessel Filling Variables
                processing_parameter=processing_parameter, verbose=True)

# processing_parameter = vf.default_fill_vessels_processing_parameter.copy()
# processing_parameter.update(size_max = 1000,
#                             size_min = 'fixed',
#                             axes = all,
#                             overlap = 100);

# processing_parameter['processes'] = 1

# vf.fill_vessels(source, sink,
#                 resample=2, threshold=0.5, cuda=None,
#                 processing_parameter=processing_parameter, verbose=True)

# save filled image here
io.convert_files(sink, extension='tif', processes=None, verbose=True);


# Graphing

In [ ]:
sink   = ws.filename('binary', postfix='arteries_filled_threshold' + name);
io.convert_files(sink, extension='tif', processes=None, verbose=True);


In [ ]:
binary   = ws.filename('binary', postfix='arteries_filled_threshold' + name);
skeleton = ws.filename('skeleton')

skl.skeletonize(binary, sink=skeleton, delete_border=True, verbose=True);


In [ ]:
graph_raw = gp.graph_from_skeleton(ws.filename('skeleton'), verbose=True)
#graph_raw.save(ws.filename('graph', postfix='raw'))


In [ ]:
coordinates = graph_raw.vertex_coordinates();
radii, indices = mr.measure_radius(ws.filename('binary', postfix='arteries_filled' + name), coordinates,
                                   value=0, fraction=None, max_radius=150,
#                                   value=None, fraction=0.8, max_radius=150,
                                   return_indices=True, default=-1, verbose=True);
graph_raw.set_vertex_radii(radii)


In [ ]:
graph_raw.save(ws.filename('graph', postfix='raw'))


In [ ]:
# Graph cleaning
graph_cleaned = gp.clean_graph(graph_raw,
                               vertex_mappings = {'coordinates'   : gp.mean_vertex_coordinates,
                                                  'radii'         : np.max,
                                                  'artery_binary' : np.max,
                                                  'artery_raw'    : np.max},
                               verbose=True)


In [ ]:
# Save cleaned graph

graph_cleaned.save(ws.filename('graph', postfix='cleaned'))
#graph_cleaned = grp.load(ws.filename('graph', postfix='cleaned'));


In [ ]:
# Graph reduction

def vote(expression):
  return np.sum(expression) >= len(expression) / 1.5;

graph_reduced = gp.reduce_graph(graph_cleaned, edge_length=True,
                          edge_to_edge_mappings = {'length' : np.sum},
                          vertex_to_edge_mappings={'artery_binary' : vote,
                                                   'artery_raw'    : np.max,
                                                   'radii'         : np.max},
                          edge_geometry_vertex_properties=['coordinates', 'radii', 'artery_binary', 'artery_raw'],
                          edge_geometry_edge_properties=None,
                          return_maps=False, verbose=True)


In [ ]:
# Save reduced graph

graph_reduced.save(ws.filename('graph', postfix='reduced'))
#graph_reduced = grp.load(ws.filename('graph', postfix='reduced'));


In [ ]:
# Graph largests component

graph = graph_reduced.largest_component()
graph.save(ws.filename('graph', postfix='reduced_largest'))


In [ ]:
graph_reduced = grp.load(ws.filename('graph', postfix='reduced_largest'));
verticies = graph_reduced.vertex_coordinates()
connectivity = graph_reduced.edge_connectivity()
radii = graph_reduced.vertex_radii()


In [ ]:
print(connectivity)
print(len(connectivity))
print(verticies)
print(radii)
print(len(radii))


In [ ]:
print(max(radii))


In [ ]:
import pickle
pickle.dump(connectivity, open(path + "connectivity_reduced.p", "wb" ) )
pickle.dump(verticies, open(path + "verticies_reduced.p", "wb" ) )
pickle.dump(radii, open(path + "radii_reduced.p", "wb" ) )
